# Lab 3: Save and Load Models

In this notebook, we'll complete the final step of our PyTorch workflow — saving a trained model to disk and loading it back for future use. This is essential for deploying models or continuing work later.

## Install Dependencies

Run this cell to install the required libraries. We need PyTorch for model operations and `pathlib` (built-in) for file path handling.

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install matplotlib

## Setup from Labs 1 and 2

Before we can save a model, we need a trained model. Let's quickly recreate everything from Labs 1 and 2.

**From Lab 1 (Data and Model Building):**
- Create synthetic data using `y = 0.4 * X + 0.1`
- Split into 80% training and 20% testing sets
- Define `LinearRegressionModel` class with learnable `weight` and `bias` parameters

**From Lab 2 (Training Loop):**
- Set up loss function (`nn.L1Loss`) and optimizer (`SGD` with lr=0.01)
- Train for 100 epochs using the 5-step training loop
- Model learns to approximate `weight ≈ 0.4` and `bias ≈ 0.1`

![Linear Model](https://raw.githubusercontent.com/poridhiEng/lab-asset/7008e578e0c9c57813d1b267134700911793d762/tensorcode/Deep-learning-with-pytorch/LinearRegression/lab-02/images/linear-model.svg)

The model takes input X, multiplies it by weights, adds bias, and outputs predictions. After training, it should discover `weight ≈ 0.4` and `bias ≈ 0.1`.

In [ ]:
import torch
import torch.nn as nn
from pathlib import Path

torch.manual_seed(42)

# Target parameters (what we want the model to learn)
weight = 0.4
bias = 0.1

# Create data
X = torch.arange(0, 1, 0.02).unsqueeze(dim=1)
y = weight * X + bias

# Train/test split (80/20)
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

# Model definition
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(1), requires_grad=True)
        self.bias = nn.Parameter(torch.randn(1), requires_grad=True)

    def forward(self, x):
        return self.weight * x + self.bias

# Create model instance
model = LinearRegressionModel()

print(f"Training samples: {len(X_train)}, Test samples: {len(X_test)}")
print(f"Initial parameters: weight={model.weight.item():.4f}, bias={model.bias.item():.4f}")
print(f"Target parameters:  weight={weight}, bias={bias}")

## Train the Model

We'll quickly train the model using the 5-step training loop from Lab 2.

In [ ]:
# Create loss function and optimizer
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Training loop
epochs = 100

for epoch in range(epochs):
    model.train()
    
    # 1. Zero gradients
    optimizer.zero_grad()
    
    # 2. Forward pass
    y_pred = model(X_train)
    
    # 3. Calculate loss
    loss = loss_fn(y_pred, y_train)
    
    # 4. Backward pass
    loss.backward()
    
    # 5. Update parameters
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d} | Loss: {loss:.4f}")

print(f"\nTraining complete!")
print(f"Learned parameters: weight={model.weight.item():.4f}, bias={model.bias.item():.4f}")
print(f"Target parameters:  weight={weight}, bias={bias}")

## 1. Understanding state_dict()

The `state_dict()` is a Python dictionary that maps each layer/parameter name to its tensor value. It contains all the learnable parameters of your model — in our case, just `weight` and `bias`.

This is what we save to disk and load back later. By saving only the `state_dict()` (not the entire model), we keep our saved files small and portable.

In [ ]:
# View the state dict
print("Model state_dict():")
print(model.state_dict())

print("\nBreaking it down:")
for param_name, param_value in model.state_dict().items():
    print(f"  {param_name}: {param_value.item():.4f}")

## 2. Create a Models Directory

It's good practice to organize saved models in a dedicated folder. This keeps your project clean and makes it easy to find model files later.

We use Python's `pathlib.Path` for cross-platform compatibility — it handles file paths correctly on Windows, Mac, and Linux.

In [ ]:
# Create models directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

print(f"Model directory created: {MODEL_PATH}")

## 3. Define the Save Path

We use `.pth` or `.pt` extension for PyTorch model files. This is a convention (not required) that helps identify PyTorch models at a glance.

Common naming patterns: `model_name.pth`, `model_v1.pt`, `best_model.pth`

In [ ]:
# Define model filename and full path
MODEL_NAME = "linear_regression_model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Model will be saved to: {MODEL_SAVE_PATH}")

## 4. Save the Model

We save only the `state_dict()`, not the entire model object. This is the **recommended approach** because:

- **Portable**: Works even if you rename classes or move files
- **Flexible**: Can load into modified model architectures (if compatible)
- **Smaller files**: Only stores the essential parameter values

The alternative (`torch.save(model, path)`) saves the entire model but can break if your code structure changes.

In [ ]:
# Save the model state dict
torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH)

print(f"Model saved to: {MODEL_SAVE_PATH}")

## 5. Verify the File Exists

Always verify that your model was saved successfully before moving on. A quick check now can save debugging time later.

We'll confirm the file exists and list all files in the models directory.

In [ ]:
# Check if file exists
print(f"File exists: {MODEL_SAVE_PATH.exists()}")

# List files in models directory
print(f"Files in {MODEL_PATH}: {list(MODEL_PATH.iterdir())}")

## 6. Load the Model

Loading a saved model involves three steps:

1. **Create a new model instance** — This starts with random parameters (just like when we first built the model)
2. **Load the saved state dict** — Use `torch.load()` to read the `.pth` file from disk
3. **Apply to the model** — Use `load_state_dict()` to replace random parameters with trained ones

**Important**: You must have access to the model class definition (`LinearRegressionModel`) to load the model. The saved file only contains parameter values, not the model architecture.

In [ ]:
# Step 1: Create a new model instance (with random parameters)
loaded_model = LinearRegressionModel()
print(f"Before loading (random params): {loaded_model.state_dict()}")

In [ ]:
# Steps 2 & 3: Load the saved state dict into the model
loaded_model.load_state_dict(torch.load(f=MODEL_SAVE_PATH))
print(f"After loading (trained params): {loaded_model.state_dict()}")

Notice how the parameters changed from random values to the trained values after loading! The `load_state_dict()` function replaced all the random parameters with our saved trained parameters.

## 7. Verify Predictions Match

The ultimate test: the loaded model should produce **identical predictions** to the original trained model. If predictions match, we know the save/load process preserved all the learned parameters correctly.

We use `torch.allclose()` to compare tensors — it returns `True` if all values are equal (within a small tolerance for floating-point precision).

In [ ]:
# Original model predictions
model.eval()
with torch.inference_mode():
    original_preds = model(X_test)

# Loaded model predictions
loaded_model.eval()
with torch.inference_mode():
    loaded_preds = loaded_model(X_test)

# Compare predictions
predictions_match = torch.allclose(original_preds, loaded_preds)
print(f"Predictions match: {predictions_match}")

In [ ]:
# Visual comparison
print("Original model predictions (first 5):")
print(original_preds[:5].squeeze())

print("\nLoaded model predictions (first 5):")
print(loaded_preds[:5].squeeze())

print("\nActual values (first 5):")
print(y_test[:5].squeeze())

All predictions are identical! This confirms that our model was saved and loaded correctly. The loaded model behaves exactly like the original trained model.

## Summary

In this lab, we learned how to:

1. **Understand `state_dict()`** — the dictionary containing all model parameters
2. **Save a trained model** using `torch.save(model.state_dict(), path)`
3. **Load a model** by creating a new instance and calling `load_state_dict(torch.load(path))`
4. **Verify predictions match** using `torch.allclose()`

## Complete PyTorch Workflow

Congratulations! You've completed the full PyTorch workflow across three labs:

1. **Lab 1: Data and Model Building** — Created data and built a model with random parameters
2. **Lab 2: Training Loop** — Implemented the 5-step training loop to learn `weight ≈ 0.4` and `bias ≈ 0.1`
3. **Lab 3: Save and Load** — Saved the trained model and loaded it back for future use

These fundamentals apply to all PyTorch projects, whether you're building simple linear regression or complex deep learning models!